In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import mean_squared_error
import torch
from torch import nn
from torch.utils.data import TensorDataset, DataLoader
import torch.optim as optim

In [27]:
# 데이터 로드
data = pd.read_csv(r"C:\Gachon\3-1\인공지능개론\diabetes.csv")

data = data.drop('Outcome', axis = 1)

# 입력과 타겟 분리
X = data.drop('BMI', axis=1).values
y = data['BMI'].values.astype(np.float32)

# 정규화
scaler = StandardScaler()
X = scaler.fit_transform(X)

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train.shape, X_test.shape, y_train.shape, y_test.shape, 

((614, 7), (154, 7), (614,), (154,))

In [29]:
# TensorDataset으로 래핑
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)

In [31]:


# 회귀 모델 정의
class RegressionModel(nn.Module):
    def __init__(self):
        super(RegressionModel, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(7, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1)
        )

    def forward(self, x):
        return self.model(x)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = RegressionModel().to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [33]:
# 학습 루프
model.train()
for epoch in range(50):
    total_loss = 0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        output = model(X_batch)
        loss = criterion(output, y_batch)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {total_loss / len(train_loader):.4f}")

Epoch 1, Loss: 1046.0974
Epoch 2, Loss: 996.1867
Epoch 3, Loss: 909.5416
Epoch 4, Loss: 747.8691
Epoch 5, Loss: 563.1932
Epoch 6, Loss: 352.2857
Epoch 7, Loss: 195.7121
Epoch 8, Loss: 107.9185
Epoch 9, Loss: 88.5818
Epoch 10, Loss: 76.7130
Epoch 11, Loss: 72.6394
Epoch 12, Loss: 71.3213
Epoch 13, Loss: 68.2213
Epoch 14, Loss: 67.1987
Epoch 15, Loss: 63.6721
Epoch 16, Loss: 66.6822
Epoch 17, Loss: 62.8231
Epoch 18, Loss: 58.9289
Epoch 19, Loss: 58.2302
Epoch 20, Loss: 57.3656
Epoch 21, Loss: 61.7461
Epoch 22, Loss: 57.6208
Epoch 23, Loss: 53.7965
Epoch 24, Loss: 52.8959
Epoch 25, Loss: 51.3746
Epoch 26, Loss: 50.2842
Epoch 27, Loss: 50.3363
Epoch 28, Loss: 49.6990
Epoch 29, Loss: 49.2035
Epoch 30, Loss: 47.8142
Epoch 31, Loss: 55.3722
Epoch 32, Loss: 45.8997
Epoch 33, Loss: 45.6517
Epoch 34, Loss: 46.2613
Epoch 35, Loss: 44.4897
Epoch 36, Loss: 45.1141
Epoch 37, Loss: 43.7256
Epoch 38, Loss: 43.0069
Epoch 39, Loss: 43.6824
Epoch 40, Loss: 42.5015
Epoch 41, Loss: 42.3271
Epoch 42, Loss: 

In [35]:
# 평가
model.eval()
preds, actuals = [], []
with torch.no_grad():
    for X_batch, y_batch in test_loader:
        X_batch = X_batch.to(device)
        outputs = model(X_batch).cpu().numpy()
        preds.extend(outputs)
        actuals.extend(y_batch.numpy())

mse = mean_squared_error(actuals, preds)
print(f"Test MSE: {mse:.4f}")

Test MSE: 65.1278
